## Diego u0283930
### Homework 3
[Robots file](https://news.ycombinator.com/robots.txt)

## Part 1

In [ ]:
from bs4 import BeautifulSoup as bs
import urllib.request
import time
import calendar
import pandas as pd
import csv
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import numpy as np

In [ ]:
# get first num_pages
def get_html_pages(num_pages):
    html = ""
    for i in range (num_pages):
        url = "https://news.ycombinator.com/news?p=" + str(i+1)
        tmp = ""
        with urllib.request.urlopen( url ) as response:
            tmp = response.read()
            html = html + tmp.decode( 'utf-8' )
            print(f"scraping {url}")
            time.sleep(1)
        
    # save the file
    name = "hacker_news.html"
    with open( name, 'w' ) as new_file:
        new_file.write(html)

get_html_pages(5)

In [ ]:
# Create the soup from local file
file = "hacker_news.html"
with open (file, 'r') as f:
    html = str(f.readlines())

soup = bs( html, 'html.parser' )

### Rank

In [ ]:
ranks = soup.select(".rank")
ranks = [int(tag.string[:-1]) for tag in ranks] # for academic practice. Could just use a loop 1 - 150

### Length of Title

In [ ]:
titles = soup.select(".titlelink")
titles_length = [ len(tag.get_text()) for tag in titles ]
assert(len(titles_length) == len(ranks))

### Age in hours

In [ ]:
def get_converted_time(age):
    time = age[0]
    unit = age[1]
    if "minute" in unit:
        return time / 60
    elif "hour" in unit:
        return time
    elif "day" in unit:
        return time * 24
    else:
        return -1

In [ ]:
time_since_post = []
ages = soup.find_all( "span", class_="age" )
for el in ages:
    age = el.get_text()
    age = age.split()
    ints = [str(integer) for integer in age[0]]
    age[0]  = "".join(ints)
    age[0]  = float(age[0]) 
    age[0] = get_converted_time(age)
    time_since_post.append(age[0])

### Points

In [ ]:
sub = soup.select(".subtext")
points = []
for tag in sub:
    tmp = tag.get_text().split()[2:4]
    # print(tmp)
    if tmp[1] == "points":
        points.append(int(tmp[0]))
    else:
        points.append(1)

assert(len(titles_length) == len(ranks) and len(titles_length) == len(time_since_post) and len(time_since_post) == len(points))

### Number of comments

In [ ]:
comments = []
for tag in sub:
    tmp = tag.get_text().split()
    # print(tmp)
    if tmp[-1] == "comments" or tmp[-1] == "comment":
        comments.append(int(tmp[-2]))
    else:
        comments.append(0)

assert(len(titles_length) == len(ranks) and len(titles_length) == len(time_since_post) and len(time_since_post) == len(points) and len(points) == len(comments))

In [ ]:
data = {"rank":ranks,
        "len_titles":titles_length,  
        "time_since_post":time_since_post, 
        "points":points, 
        "comments":comments
        }
df = pd.DataFrame(data)

In [ ]:
df.to_csv("data.csv", index=False )
df_CSV = pd.read_csv("data.csv")
df_CSV.reindex(columns=data.keys())

## Part 2

### 1

In [ ]:
plot_data = df_CSV[["rank", "time_since_post", "points"]]
data_ols = sm.ols(formula="rank ~ time_since_post + points", data=plot_data ).fit()

hours_and_points = data_ols.params["Intercept"] + (data_ols.params["points"] * df["points"]) + (data_ols.params["time_since_post"] * df["time_since_post"])
hours_and_points
df["hours_and_points"] = hours_and_points

with plt.xkcd():
    plt.scatter(x=df[["hours_and_points"]], y=df["rank"], color="purple")
    plt.plot(df["hours_and_points"],data_ols.predict(), label="best fit", linewidth=1, color="lightblue")
    plt.xlabel("hours since post and points")
    plt.ylabel("Rank")
    plt.legend()


In [ ]:
data_ols.summary()
# R-squared:	                0.394
# coef:         time_since_post	3.2406
#               points	        -0.0912	

# p-val         time_since_post 0.000	
#               points          0.000

### 2

In [ ]:
# Checking the ols of all columns
df_CSV = pd.read_csv("data.csv")
df_CSV.reindex(columns=data.keys())
data_ols = sm.ols(formula="rank ~ len_titles + points + time_since_post + comments", data=data ).fit()
data_ols.summary()
# R-squared:	                0.397
# coef:         len_titles	    0.1011
#               points	        -0.0963
#               time_since_post 3.2550
#               comments        0.0154

# p-val:        len_titles	    0.496
#               points	        0.000		
#               time_since_post 0.000
#               comments        0.593

### 3


In [ ]:
# compare the rank to the time since post
data_ols = sm.ols(formula="rank ~ time_since_post", data=data ).fit()
data_ols.summary()
# R-squared:	                0.277
# coef:       	time_since_post 2.1901

# p-val:      	time_since_post 0.000

In [ ]:
matrix = df.corr()
labels = df.keys()
with plt.xkcd():
    grid = plt.imshow(matrix, cmap="tab20c", vmax= 1, vmin= -1)
    plt.colorbar(grid, shrink=.8)
    plt.xticks(np.arange(len(labels)), labels=labels, rotation = 90)
    plt.yticks(np.arange(len(labels)), labels=labels)

In [ ]:
plot_data = df[['rank', 'points', 'len_titles', 'time_since_post', "comments"]]
with plt.xkcd():
    pd.plotting.scatter_matrix(plot_data, figsize=(10, 10), diagonal='kde', color="grey")
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)

## Write up
It appears there is a linear relationsip between age and rank. points and comments, a relationship between points and time. Age appears to be the biggest contributing factor to where a post ranks. Rank compared to lenghth of titles is a terrible comparison, the data is all over the place.


The r squared vals are low, so it appears that none of this stuff is really correlated. The best is age. But, they are all bad variables.
This data will change depending on source file and time scraped


What our regressions tell us about being on the front page is that it helps to have a high score and to have not been there for very long


## Part 3

In [ ]:
df["is_on_front_page"] = [int(row <= 30) for row in df["rank"]] # add a 1 to rows on front page, 0 otherwise
df

In [ ]:
df["score"] = ((df["points"] -1) ** .8 / (df["time_since_post"] + 2) ** 1.8)
# pd.set_option("display.max_rows", None, "display.max_columns", None) #display full dataframe
df

In [ ]:
with plt.xkcd():
    fig = plt.figure(figsize=(10,4))
    ax = fig.add_axes([1,1,1,1])
    ax.set_title("Front page predicted\nStories above line predicted to be on front page")
    ax.set_xlabel("rank")
    ax.set_ylabel("calculated score")
    df_front = df[df["is_on_front_page"] == 1] #create new data frames with 1 and 0
    df_not_front = df[df["is_on_front_page"] == 0]
    ax.scatter(x=df_front["rank"], y=df_front["score"], label="on front page")
    ax.scatter(x=df_not_front["rank"], y=df_not_front["score"], label="not on front page", c="grey")
    sorted = df["score"].sort_values(ascending=False)
    cutoff = sorted.iloc[29]
    ax.axhline(y=cutoff, color="purple") #cutoff score
    ax.legend()    


In [ ]:
with plt.xkcd():
    fig = plt.figure(figsize=(10,4))
    ax = fig.add_axes([1,1,1,1])
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Front Page")

    logit_res = sm.logit(formula="is_on_front_page ~ score", data=df).fit()
    results = logit_res.predict()

    logistic_sigmoid_func = lambda x: 1 / ( 1 + np.exp(-x) )
    ax.scatter(x=df["score"], y=df["is_on_front_page"])   
    logit_param = dict(logit_res.params)

    xs = np.linspace( df["score"].min(), df["score"].max(), 1000 )
    ys = logistic_sigmoid_func( logit_param["Intercept"] + logit_param["score"] * xs )

    fifty_percent = np.interp(0.5, ys, xs)

    ax.plot( xs, ys, color="Black") # Smushed line
    ax.plot([ fifty_percent, fifty_percent], [0,1], "r")